## TDA@YSDA

### Homework 2

### Задача 1: спектры Лапласианов (3 балла)

В этой задаче требуется найти спектры Лапласианов Ходжа графа.

Для этого предлагается самостоятельно реализовать наивный алгоритм нахождения операторов ко-границ.

Для проверки будет генерироваться случайный граф (из модели Эрдеша-Реньи – то есть, наличие любого ребра в графе – это бернуллиевская случайная величина с вероятностью успеха $p$, не зависящая от других ребер).

Гарантируется, что **граф связный**

Граф будет подаваться на вход вашей функции в виде **списка ребер**.

In [ ]:
import networkx as nx

g = nx.gnp_random_graph(n = 7, p = 0.4, seed = 42)

nx.draw(g)

print('Edges list:', g.edges())

#### Шаг 1: найти клики

Напишите функцию `find_cliques()`, которая по данному на вход списку ребер в таком формате:

`edges = [(0, 1), (0, 2), (1, 2), (0, 3)]`

– для графа на N вершинах это список упорядоченных неповторяющихся пар вершин (вершины просто занумерованы числами от 0 до N-1), образующих ребро –

**найдет** все клики (полные подграфы) в данном графе,

и вернет список списков упорядоченных кортежей вершин, образующих клики:

`simplices = [[(0,), (1,), (2,), (3,)], 
              [(0, 1), (0, 2), (1, 2), (0, 3)],
              [(0, 1, 2)]]`
              
Это можно сделать и средствами стандартной библиотеки Python, но вообще можете использовать любые библиотеки.

In [ ]:
# your code here

#### Шаг 2: построить операторы ко-границ

Зная весь кликовый комплекс графа, можно построить операторы ко-границ.

В нашем случае это просто транспонированные (сопряженные) операторы границ, так что можно строить операторы, с которыми вы уже знакомы из первой половины курса.

В нашем примере мы имеем:

`simplices = [[(0,), (1,), (2,), (3,)], 
              [(0, 1), (0, 2), (1, 2), (0, 3)],
              [(0, 1, 2)]]`

4 0-симплекса (вершины), 4 1-симплекса (ребра), 1 2-симплекс (треугольник).

Внутри своего списка все k-симплексы занумерованы. Тогда, например, оператор границы $\partial_2$:

$\partial_2 ((0,1,2)) = (\hat{0},1,2) - (0,\hat{1},2) + (0,1,\hat{2}) = (1,2) - (0,2) + (0,1)$

имеет вид

$\partial_2 = 
\begin{matrix}(0,1) \\ (0,2) \\ (1,2) \\ (0,3)\end{matrix}
\overset{(0,1,2)}{\begin{pmatrix} 1 \\ -1 \\ 1 \\ 0\end{pmatrix}}.$

Так можно построить матрицы операторов границы:

1. инициализируем ее как матрицу из нулей размера (# (k-1)-симплексов)x(# k-симплексов)
2. для каждого k-симплекса (в списке k-симплексов) проходимся по списку всех (k-1)-симплексов, проверяем, являются ли они подмножеством данного k-симплекса, и если да – то исключением какого его элемента они даются, четного или нечетного – в зависимости от этого ставим +1 или -1 в матрицу на соответствующем месте

Соответствующий оператор ко-границы $\delta_2 : L^2_\wedge(K_2)\to L^2_\wedge(K_{3})$
$= \partial^T_2$ – это просто транспонированный $\partial_2$.

k-й лапласиан Ходжа – это:

$$\Delta^{~}_k = \delta^{~}_{k-1}\delta^T_{k-1} + \delta^T_{k}\delta^{~}_{k}$$

In [ ]:
# your code here

#### Шаг 3: найдите спектры Лапласианов

Лапласианы – это эрмитовы матрицы, потому для нахождения их собственных чисал можно пользоваться, например, `numpy.linalg.eigh`.

**Ответом** на задачу является матрица из собственных значений всех ненулевых Лапласианов Ходжа данного на вход графа. 

$$A = \begin{pmatrix}\lambda^{(0)}_\min & ... \\
\lambda^{(1)}_\min & ... \\ 
... & ...\\
\lambda^{(m)}_\min & ...
\end{pmatrix}$$

Т.е. k-ая строчка матрицы = спектр k-ого Лапласиана, записанный **в порядке возрастания собственных чисел** (они не могут быть неотрицательными, потому первые несколько из них с численной точностью равны нулю, остальные – положительны). 

Размерность k-ого Лапласиана – это (# k-симплексов)x(# k-симплексов) – потому число столбцов матрицы = максимальная из размерностей Лапласианов, к спектрам меньших размерностей **дописываем справа нули**.

Так, у графа в нашем примере это 

$$
\begin{pmatrix}\lambda^{(0)}_{0=\min} & \lambda^{(0)}_1 & \lambda^{(0)}_2 & \lambda^{(0)}_{4=\max} \\
\lambda^{(1)}_{0=\min} & \lambda^{(1)}_1 & \lambda^{(1)}_2 & \lambda^{(1)}_{4=\max} \\ 
\lambda^{(2)}_{0} & 0 & 0 & 0\\
\end{pmatrix}
$$

Матричные элементы давайте **округлять до 3 знаков после запятой** – это можно сделать `numpy.round(A, 3)`.

In [ ]:
# your code here

### Задача 2: локализация источника (на пути к HodgeNet) (2 балла)

В этой задаче мы будем искать источник потока на графе, что делается в статье _[HodgeNet: Graph Neural Networks for Edge Data](https://arxiv.org/abs/1912.02354)_ при помощи простой нейросети. 

Итак, наш граф задан набором из $N$ вершин $\mathcal{V}=\{0,1,\dots,N-1\}$, и $E$ ребер $\mathcal{E}=\{(i,j)\}$.
Как и в прошлой задаче, ребро задается парой (номеров) вершин, упорядоченной по возрастанию.

**Потоком** называется функция $f$ на ребрах графа $f:\mathcal{E}\to \mathbb{R}^E$. Можно думать, что она "на вход" принимает пару вершин и возвращает вещественное число, при этом 

$$f(i,j)=-f(j,i)$$

– если порядок вершин в паре поменять, она поменяет знак (таким образом мы следим за ориентацией ребер).

Если мы договоримся, что ребра задаются упорядоченными (по возрастанию) парами (номеров) вершин, то поток можно записать просто как вектор $\mathbf{f}$ размера $E$ (для удобства будем следить за размерами векторов и матриц в этой задаче) значений на каждом ребре 

$$\mathbf{f}_{E} = \begin{matrix}(0,1) \\ (0,2) \\ (1,2) \\ (0,3)\end{matrix}
\begin{pmatrix} 1 \\ -2 \\ 3.14 \\ 0\end{pmatrix}$$

– для примера графа из задачи 1.

Итак, в статье утверждается, что если построить матрицу $B_{N\times E}$ инцидентности вершин ребрам:

$$B_{ve} = \begin{cases}
-1,~~\text{если }v\text{ – "начало" ребра }e\\
+1,~~\text{если }v\text{ – "конец" ребра }e\\
0~~~~~\text{ – иначе}
\end{cases}$$

и построить из нее **1-Лапласиан Ходжа**

$$ \Delta_1 = B^TB,$$

то будет справедливо **разложение Ходжа:**

$$\mathbb{R}^E=\text{ker}(\Delta_1)\otimes\text{im}(B^T) \tag{1}$$

(то есть $(B^T)_{E\times N}$ – есть не что иное как градиент), то есть всякий поток раскладывается на две ортогональные компоненты:

$$\forall \mathbf{f}\in \mathbb{R}^E:~~\mathbf{f}=\mathbf{h}+ B^T\,\varphi$$

где $\mathbf{h}_E$ – **гармоническая** компонента ($\in\text{ker}(\Delta_1)$), а $\varphi$ – **потенциал**, то есть функция на вершинах (его можно просто записывать как вектор $\varphi_V$ из $N$ чисел).

Именно потенциал ответственен за **источники**. Источником называется такая вершина, в которой потенциал положителен (из нее потенциальное поле "вытекает").

_Однако, нужно сказать, что в этом месте в статье имеется неточность – мы с вами знаем, что $\text{ker}(\Delta_1)\otimes\text{im}(B^T)$ составляют только $\text{ker}(\text{rot})$ – ядро ротора (безвихревые поля)! До полного $\mathbb{R}^E$ в разложении $(1)$ не хватает $\text{im}(\text{rot}^*)$. Потому в данной задаче мы ограничимся рассмотрением безвихревых полей, то есть таких, что $\in\text{ker}(\text{rot})$_

Итак, имея поток $\mathbf{f}$ (гарантируется, что он безвихревой), мы можем **найти источник**, избавившись от гармонической компоненты $\mathbf{h}_f$, и решив уравнение:

$$B^T\varphi = \mathbf{f}-\mathbf{h}_f$$

Конечно, матрица $B$ может быть не полного ранга (и вообще не квадратная), потому решение не единственно! Но можно найти решение наименьшей нормы, взяв **псевдообратную** матрицу $(B^T)^+ = (BB^T)^{-1}B$ к матрице $B^T$.

Тогда будем иметь

$$\tilde{\varphi} = (B^T)^+ \, (\mathbf{f}-\mathbf{h}_f)$$

– "наилучший" (в смысле наименьшей нормы) потенциал $\tilde{\varphi}$ такой, что его образом $B^T\tilde{\varphi}$ является $\mathbf{f}$ – с точностью до гармонической части $\mathbf{h}_f$.

Чтобы найти "главный" источник, найдем вершину (компоненту вектора $\tilde{\varphi}$), в которой значение потенциала максимально:

$$i_\text{source} = \text{arg}\max_i \tilde{\varphi}_i$$

Собственно, в статье HodgeNet эта задача (в несколько модифицированном виде – находится не сам главный источник, а подмножество вершин, которому он принадлежит) решается нейросетью, которой на вход при обучении подаются результаты действия **степеней** 1-Лапласиана на входной поток

$$\mathbf{f},~(\Delta_1/\lambda_1)\,\mathbf{f},~(\Delta_1/\lambda_1)^2\,\mathbf{f},\dots$$

где Лапласиан отнормирован на $\lambda_1$ – свое старшее собственное число (это нужно, чтобы норма вектора при действии не увеличивалась и на вход нейросети поступали небольшие числа). Действие лапласиана на вектор потока $\mathbf{f}$ зануляет его гармоническую компоненту $\mathbf{h}_f$ и оставляет только потенциальную. По тому, какие результаты дает действие разных степеней $(\Delta_1/\lambda_1)$ на эту потенциальную часть, нейросеть учится определять ее источник. _(Кстати, такое действие степенями определенного оператора называется применением фильтра к сигналу)_.

Итак, вам будет **дано**:

1) граф, заданный списком ребер, в формате `edges = [(0, 1), (0, 2), (1, 2), (0, 3)]`

2) поток на нем – в формате `flow =  [1, -2, 3.14, 0]`

(гарантируется, что поток безвихревой).

Ваша **задача**

1) отделить от сигнала его гармоническую компоненту

2) псевдообращением матрицы градиента $B^T$ получить наилучшее приближение к потенциалу $\tilde{\varphi}$

3) вернуть номер вершины с наибольшим значением потенциала

**Подсказка:** матрица $B$ – это оператор границы $\partial_1$, построение которого вы должны были реализовать в предыдущей задаче.

In [ ]:
# your code here